Convert the source csv file into two csv files, for cortical and subcortical points respectively, with the following columns: roi label, subject ID, x, y, z.

---

Dataset specific info

In [ ]:
src_csv = '../data/Final_Cortical_DES_041121SN_ID.csv'

In [ ]:
out_csv_cortical = '../data/des_points_cortical.csv'
out_csv_subcortical = '../data/des_points_subcortical.csv'

Definition of labels for ROIs

In [ ]:
cortical_lab = {}
cortical_lab['speech_arrest'] = 4
cortical_lab['semantic_paraphasia'] = 5
cortical_lab['articulary_disorders'] = 6
cortical_lab['phonological_aphasia'] = 7
cortical_lab['anomia'] = 8
cortical_lab['amodal_anomia'] = 9
cortical_lab['motor_dystonia'] = 10
cortical_lab['movement_arrest'] = 11
cortical_lab['face_movement'] = 12
cortical_lab['visual'] = 13
cortical_lab['sensorial'] = 14
cortical_lab['mentalizing'] = 15
cortical_lab['vocalisation'] = 17
cortical_lab['visuo_spatial'] = 18
cortical_lab['hypophonia'] = 19
cortical_lab['perseverations'] = 20
cortical_lab['complex_behavioral_reaction'] = 22
cortical_lab['self_evaluation'] = 23
cortical_lab['eyes_movements'] = 24
cortical_lab['reading'] = 25
cortical_lab['echolalia'] = 26


In [ ]:
subcortical_lab = {}
subcortical_lab['motor'] = 29
subcortical_lab['sensorial'] = 30
subcortical_lab['speech_arrest'] = 31
subcortical_lab['movement_arrest'] = 32
subcortical_lab['verbal_arrest'] = 33
subcortical_lab['anomia'] = 34
subcortical_lab['semantic_anomia'] = 35
subcortical_lab['phonological_anomia'] = 36
subcortical_lab['involuntary_movements'] = 37
subcortical_lab['visual'] = 38
subcortical_lab['visuo_spatial'] = 39
subcortical_lab['verbal_apraxia'] = 40
subcortical_lab['amodal_anomia'] = 41
subcortical_lab['reading'] = 42
subcortical_lab['eyes_movements'] = 43
subcortical_lab['psychiatric_reactions'] = 44
subcortical_lab['mentalizing'] = 45
subcortical_lab['acoustic_responses'] = 46
subcortical_lab['vertigo'] = 47
subcortical_lab['perseveration'] = 48

In [ ]:
num_sub = 307

In [160]:
bad_sub = ['KAGKNIKO']

---

In [129]:
import csv
import re
import numpy as np

Load the CVS file with comma separation

In [ ]:
csv_fd = open(src_csv)
csv_reader = csv.reader(csv_fd)
csv_header = next(csv_reader)
csv_rows = []
for row in csv_reader:
    csv_rows.append(row)
csv_fd.close()

Check the correspondence between labels and columns

In [ ]:
for lab in list(cortical_lab.keys()):
    print(lab,csv_header[cortical_lab[lab]])

In [ ]:
for lab in list(subcortical_lab.keys()):
    print(lab,csv_header[subcortical_lab[lab]])

Extract 3D points and write to file

In [183]:
def extract_3d_points(roi, out_csv, num_sub, bad_sub):
    with open(out_csv, 'w') as out_csv_fd:
        sep = ","
        for lab in list(roi.keys()):
            for row in range(num_sub):
                roi_col = roi[lab]
                sub = csv_rows[row][0]
                if not sub in bad_sub:
                    pts = re.sub("\([A-Z,a-z,0-9,\-,+,\ ,/,\',à]*\)|[A-Z,a-z]|\'|;|:|\+|\/", ' ', csv_rows[row][roi_col])
                    pts = re.sub("\(|\)", ' ', pts)
                    pts = [x for x in pts.split(' ') if x]
                    if not(len(pts)%3):
                        pts_xyz = [pts[i:i+3] for i in range(0, len(pts), 3)]
                        for x,y,z in pts_xyz:
                            out_csv_fd.write("%s%s%s%s%s%s%s%s%s\n" % (lab, sep, sub, sep, x, sep, y, sep, z))
                    else:
                        print("WARNING - Value is not a 3D point")
                        print("ID: %s, LAB: %s, PTS: %s" % (sub, lab, pts))
                        print("SRC: %s" % csv_rows[row][roi_col])

In [ ]:
extract_3d_points(cortical_lab, out_csv_cortical, num_sub, bad_sub)

In [186]:
extract_3d_points(subcortical_lab, out_csv_subcortical, num_sub, bad_sub)

Points Analytics

In [151]:
def pts_stat(src_csv, roi_dict):
    csv_fd = open(src_csv)
    csv_reader = csv.reader(csv_fd)
    csv_rows = []
    for row in csv_reader:
        csv_rows.append(row)
    csv_fd.close()
    roi_lab = list(roi_dict.keys())
    roi_stat = np.zeros(len(roi_lab))
    for lab in range(len(csv_rows)):
        lab_idx = roi_lab.index(csv_rows[lab][0])
        roi_stat[lab_idx] += 1
    for lab in range(len(roi_lab)):
        print("%30s: %d" % (roi_lab[lab], roi_stat[lab]))
    print("%30s: %d" % ("TOTAL NUMBER OF POINTS", roi_stat.sum()))

In [185]:
pts_stat(out_csv_cortical, cortical_lab)

                 speech_arrest: 332
           semantic_paraphasia: 46
          articulary_disorders: 259
          phonological_aphasia: 7
                        anomia: 108
                 amodal_anomia: 79
                motor_dystonia: 123
               movement_arrest: 324
                 face_movement: 156
                        visual: 7
                     sensorial: 161
                   mentalizing: 47
                  vocalisation: 47
                 visuo_spatial: 10
                    hypophonia: 14
                perseverations: 0
   complex_behavioral_reaction: 0
               self_evaluation: 2
                eyes_movements: 8
                       reading: 1
                     echolalia: 2
        TOTAL NUMBER OF POINTS: 1733


In [187]:
pts_stat(out_csv_subcortical, subcortical_lab)

                         motor: 31
                     sensorial: 25
                 speech_arrest: 30
               movement_arrest: 60
                 verbal_arrest: 33
                        anomia: 35
               semantic_anomia: 127
           phonological_anomia: 32
         involuntary_movements: 0
                        visual: 24
                 visuo_spatial: 4
                verbal_apraxia: 24
                 amodal_anomia: 0
                       reading: 1
                eyes_movements: 5
         psychiatric_reactions: 0
                   mentalizing: 61
            acoustic_responses: 3
                       vertigo: 3
                 perseveration: 15
        TOTAL NUMBER OF POINTS: 513
